In [2]:
# DB 연결
import psycopg2

conn_str = "host=localhost dbname=postgres user=postgres password=admin port=5432"

try:
    conn = psycopg2.connect(conn_str)
    print("=== 접속 성공 =====")
except psycopg2.DatabaseError as db_err:
    print("접속오류 !!")
    print(db_err)

cur = conn.cursor() #cursor 인스턴스: 데이터 조작용 생성

=== 접속 성공 =====


In [3]:
#과제 1
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome('chromedriver.exe')
driver.get("https://www.billboard.com/charts/hot-100/")

#driver.maximize_window() # 창 최대화

elem = driver.find_element(By.CLASS_NAME, 'chart-results-list')
list_rows = elem.find_elements(By.CLASS_NAME, 'o-chart-results-list-row-container') 

#week
Wkelements = driver.find_elements_by_tag_name('p')
wk_list = []
for wkelement in Wkelements:
    wk_list.append(wkelement.text)
print(wk_list[10]) # wk


C:\Users\q\AppData\Local\Temp/ipykernel_7276/4168723780.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
C:\Users\q\AppData\Local\Temp/ipykernel_7276/4168723780.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  Wkelements = driver.find_elements_by_tag_name('p')


WEEK OF FEBRUARY 5, 2022


In [21]:
#추출 + DB 인서트
for list_row in list_rows:
    c_labels = list_row.find_elements(By.CLASS_NAME, 'c-label')
    title = list_row.find_element(By.ID, 'title-of-a-story') # 곡명

    #print(c_labels[0].text, '\t', c_labels[1].text, '\t', title.text) # 1번째 index가 순위, 2번째 index가 가수이름
    
    try:
        cur.execute("INSERT INTO public.billboard (wk, rank, title, singer) VALUES (%s,%s,%s,%s);", (wk_list[10],c_labels[0].text, title.text, c_labels[1].text))
        conn.commit()
        print("--- insert ok")
    except:
        conn.rollback()
        print("--- DB error")


1 	 Carolina Gaitan, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz & Encanto Cast 	 We Don't Talk About Bruno
--- insert ok
2 	 Adele 	 Easy On Me
--- insert ok
3 	 Glass Animals 	 Heat Waves
--- insert ok
4 	 The Kid LAROI & Justin Bieber 	 Stay
--- insert ok
5 	 Kodak Black 	 Super Gremlin
--- insert ok
6 	 Ed Sheeran 	 Shivers
--- insert ok
7 	 Gunna & Future Featuring Young Thug 	 Pushin P
--- insert ok
8 	 GAYLE 	 abcdefu
--- insert ok
9 	 Jessica Darrow 	 Surface Pressure
--- insert ok
10 	 Elton John & Dua Lipa 	 Cold Heart (PNAU Remix)
--- insert ok
11 	 Doja Cat 	 Need To Know
--- insert ok
12 	 Ed Sheeran 	 Bad Habits
--- insert ok
13 	 Justin Bieber 	 Ghost
--- insert ok
14 	 Lil Nas X & Jack Harlow 	 Industry Baby
--- insert ok
15 	 Silk Sonic (Bruno Mars & Anderson .Paak) 	 Smokin Out The Window
--- insert ok
16 	 Muni Long 	 Hrs And Hrs
--- insert ok
17 	 Dua Lipa 	 Levitating
--- insert ok
18 	 Lil Nas X 	 Thats What I Want
--- insert ok
19 	 Th

In [24]:
#조회
try:
    cur.execute("select * from billboard;") # sql 문장
    conn.commit()
    rows = cur.fetchall()
    for row in rows:
        print(row)
except:
    conn.rollback()
    print("--- DB error")

('WEEK OF FEBRUARY 5, 2022', '1', "We Don't Talk About Bruno", 'Carolina Gaitan, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz & Encanto Cast')
('WEEK OF FEBRUARY 5, 2022', '2', 'Easy On Me', 'Adele')
('WEEK OF FEBRUARY 5, 2022', '3', 'Heat Waves', 'Glass Animals')
('WEEK OF FEBRUARY 5, 2022', '4', 'Stay', 'The Kid LAROI & Justin Bieber')
('WEEK OF FEBRUARY 5, 2022', '5', 'Super Gremlin', 'Kodak Black')
('WEEK OF FEBRUARY 5, 2022', '6', 'Shivers', 'Ed Sheeran')
('WEEK OF FEBRUARY 5, 2022', '7', 'Pushin P', 'Gunna & Future Featuring Young Thug')
('WEEK OF FEBRUARY 5, 2022', '8', 'abcdefu', 'GAYLE')
('WEEK OF FEBRUARY 5, 2022', '9', 'Surface Pressure', 'Jessica Darrow')
('WEEK OF FEBRUARY 5, 2022', '10', 'Cold Heart (PNAU Remix)', 'Elton John & Dua Lipa')
('WEEK OF FEBRUARY 5, 2022', '11', 'Need To Know', 'Doja Cat')
('WEEK OF FEBRUARY 5, 2022', '12', 'Bad Habits', 'Ed Sheeran')
('WEEK OF FEBRUARY 5, 2022', '13', 'Ghost', 'Justin Bieber')
('WEEK OF FEBRUARY 5, 2

In [5]:
#과제 2
import requests
from bs4 import BeautifulSoup

start_page = int(input("찾고싶은 시작 페이지를 입력해주세요 : "))
end_page = int(input("찾고싶은 끝 페이지를 입력해주세요 : "))
pages = end_page - start_page + 1

for i in range(pages):
    page = str(start_page + i)

    url = ("https://jolse.com/category/toners-mists/1019/?page=" + page)
    result = requests.get(url)

    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

    bs_obj = BeautifulSoup(response.content, 'html.parser')
    #print(bs_obj)

    ul = bs_obj.find("ul", {"class": "prdList grid5"})
    #print(ul)

    divs = ul.find_all("div", {"class": "description"})
    #print(divs)

    for div in divs:
        # print(div)
        strong = div.find("strong", {"class": "name"}) # 링크, 상품명
        # print(strong)

        a = strong.find("a") # 링크
        href = a["href"] # 링크
        # print("링크: ",href)

        products = a.find_all("span") # 상품명
        # print(products)
        # print("상품명: ",products[2].text)

        li = div.find("li", {"class": "xans-record-"}) # 가격용
        prices = li.find_all("span") # 3개 나온다.-- 2번째에 있다.
        # print(spans)
        #llprint("가격: ",prices[1].text)

        try:
            cur.execute("INSERT INTO public.jolse (prod, link, price) VALUES (%s,%s,%s);", (products[2].text, href, prices[1].text))
            conn.commit()
            print("--- insert ok")
        except:
            conn.rollback()
            print("--- DB error")
    

가격:  USD 21.00
가격:  USD 22.00
가격:  USD 16.45
가격:  USD 37.00
가격:  USD 24.55
가격:  USD 21.00
가격:  USD 22.00
가격:  USD 16.45
가격:  USD 37.00
가격:  USD 24.55
가격:  USD 21.00
가격:  USD 22.00
가격:  USD 16.45
가격:  USD 37.00
가격:  USD 24.55


In [27]:
#조회
try:
    cur.execute("select * from jolse;") # sql 문장
    conn.commit()
    rows = cur.fetchall()
    for row in rows:
        print(row)
except:
    conn.rollback()
    print("--- DB error")

('Abib Heartleaf Calming Toner Skin Booster 210ml', '/product/abib-heartleaf-calming-toner-skin-booster-210ml/43661/category/1019/display/2/', 'USD 21.00')
('SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml', '/product/some-by-mi-propolis-b5-glow-barrier-calming-toner-150ml/42205/category/1019/display/2/', 'USD 22.00')
('Cos De BAHA Centella Facial Toner 200ml', '/product/cos-de-baha-centella-facial-toner-200ml/40098/category/1019/display/2/', 'USD 16.45')
('Haruharu WONDER Black Rice Hyaluronic Toner 300ml', '/product/haruharu-wonder-black-rice-hyaluronic-toner-300ml/37796/category/1019/display/2/', 'USD 37.00')
('Beauty of Joseon Ginseng Essence Water 150ml', '/product/beauty-of-joseon-ginseng-essence-water-150ml/34674/category/1019/display/2/', 'USD 24.55')
